In [10]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
!pip install yarl wordninja

In [51]:
!kaggle datasets download shawon10/url-classification-dataset-dmoz

100%|██████████████████████████████████████| 69.6M/69.6M [00:07<00:00, 12.4MB/s]
100%|██████████████████████████████████████| 69.6M/69.6M [00:07<00:00, 10.1MB/s]


<IPython.core.display.Javascript object>

In [244]:
import pandas as pd
from yarl import URL

df = pd.read_csv(
    "url-classification-dataset-dmoz/URL Classification.csv", index_col=0, header=None
)
df.columns = ["url", "type"]
df = df.reset_index(drop=True).dropna()
df.sample(10)

/home/khuyen/Data-science/venv/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,url,type
212798,http://www.sairegion15.org/,Arts
579328,http://www.s3host.com,Computers
805934,http://www.kreisjugendfeuerwehr-gotha.de/,Kids
28647,http://www.rotlichthits.de/,Adult
856878,http://www.flytheboat.com,Recreation
215786,http://www.qesnrecit.qc.ca/schools/olp/2mmsolv...,Arts
799537,http://videogames.yahoo.com/gamereview?cid=199...,Kids
469135,http://www.pmdm.com/,Business
142055,http://members.tripod.com/thescottstappsite/in...,Arts
1097568,http://nobelprize.org/economics/laureates/2002/,Science


<IPython.core.display.Javascript object>

In [133]:
url1 = df["url"][1545858]
url1

'http://www.geocities.com/peterfullman2001/concord_youth.html'

<IPython.core.display.Javascript object>

In [134]:
URL(url1).scheme

'http'

<IPython.core.display.Javascript object>

In [135]:
URL(url1).host

'www.geocities.com'

<IPython.core.display.Javascript object>

In [136]:
URL(url1).path

'/peterfullman2001/concord_youth.html'

<IPython.core.display.Javascript object>

In [145]:
URL(url1).parts

('/', 'peterfullman2001', 'concord_youth.html')

<IPython.core.display.Javascript object>

In [246]:
df["host"] = df.url.apply(lambda url: URL(str(url)).host)
df["parts"] = df.url.apply(lambda url: URL(str(url)).parts)

<IPython.core.display.Javascript object>

In [268]:
df.dropna(axis=0, subset=["host"], inplace=True)

<IPython.core.display.Javascript object>

In [247]:
import plotly.express as px

<IPython.core.display.Javascript object>

In [248]:
df.type.value_counts()

Arts          253840
Society       243942
Business      240176
Computers     117961
Science       110286
Recreation    106586
Sports        101328
Shopping       95270
Health         60097
Reference      58247
Games          56477
Kids           46182
Adult          35325
Home           28269
News            8989
Name: type, dtype: int64

<IPython.core.display.Javascript object>

In [253]:
import wordninja
from pipe import traverse

<IPython.core.display.Javascript object>

In [254]:
wordninja.split("angelfire")

['angel', 'fire']

<IPython.core.display.Javascript object>

In [255]:
wordninja.split("newadvent")

['new', 'advent']

<IPython.core.display.Javascript object>

In [256]:
wordninja.split("familytreemaker")

['familytree', 'maker']

<IPython.core.display.Javascript object>

In [257]:
df.host.isnull().sum()

5

<IPython.core.display.Javascript object>

In [258]:
def split_url(url: str):
    parts = url.split(".")
    return list([wordninja.split(part) for part in parts] | traverse)

<IPython.core.display.Javascript object>

In [259]:
split_url("familytreemaker.genealogy.com")

['familytree', 'maker', 'genealogy', 'com']

<IPython.core.display.Javascript object>

In [263]:
df["host"].isnull().sum()

5

<IPython.core.display.Javascript object>

In [269]:
df["host_parts"] = df["host"].apply(split_url)

<IPython.core.display.Javascript object>

In [352]:
def filter_type(df: pd.DataFrame, type_: str, column: str):
    return df.loc[df["type"] == type_, column]


def get_top_host_for_type(df: pd.DataFrame, type_: str, n=10):
    s = filter_type(df, type_, "host")
    return s.value_counts().nlargest(n)


def get_top_host_part_for_type(df: pd.DataFrame, type_: str, n=10):
    s = filter_type(df, type_, "host_parts")
    return s.explode().value_counts().nlargest(n)


def get_top_part_for_type(df: pd.DataFrame, type_: str, n=10):
    parts_to_exclude = ["/", ""]
    s = filter_type(df, type_, "parts")
    s = s.explode()
    s = s[~s.isin(parts_to_exclude)]
    s = s.value_counts().nlargest(n)
    return s


def plot_series(s: pd.Series):
    plot = px.bar(x=s.index, y=s.values)
    plot.show()

<IPython.core.display.Javascript object>

In [360]:
plot_series(get_top_part_for_type(df, "Health", 50))

<IPython.core.display.Javascript object>

In [361]:
plot_series(get_top_part_for_type(df, "Arts", 20))

<IPython.core.display.Javascript object>

In [362]:
plot_series(get_top_host_for_type(df, "Arts"))

<IPython.core.display.Javascript object>

In [363]:
plot_series(get_top_host_for_type(df, "Society"))

<IPython.core.display.Javascript object>

In [377]:
plot_series(get_top_host_for_type(df, "Computers", 20))

<IPython.core.display.Javascript object>

In [371]:
host_count = df["host"].value_counts()
host_count[host_count > 100]

www.geocities.com            50495
www.angelfire.com            20604
www.newadvent.org            13481
members.tripod.com            7968
us.imdb.com                   6619
                             ...  
www.urmc.rochester.edu         101
www.christianitytoday.com      101
www.flags.net                  101
www.pyramidpassion.co.uk       101
home.neopets.com               101
Name: host, Length: 543, dtype: int64

<IPython.core.display.Javascript object>

In [283]:
get_top_host_for_type(df, "Society")

host                           type   
www.newadvent.org              Society    13368
www.geocities.com              Society     9192
www.angelfire.com              Society     3723
members.tripod.com             Society     1531
groups.yahoo.com               Society     1438
news.bbc.co.uk                 Society     1336
en.wikipedia.org               Society     1232
familytreemaker.genealogy.com  Society     1158
www.gbgm-umc.org               Society      907
saints.sqpn.com                Society      636
dtype: int64

<IPython.core.display.Javascript object>

In [285]:
get_top_host_part_for_type(df, "Arts", 20)

host_parts  type
www         Arts    192698
com         Arts    184168
org         Arts     22974
cities      Arts     18815
geo         Arts     18792
net         Arts     15091
uk          Arts     13031
co          Arts     10487
imdb        Arts      9854
fire        Arts      8628
angel       Arts      8627
us          Arts      7469
tripod      Arts      7188
edu         Arts      6953
the         Arts      6105
a           Arts      5597
s           Arts      5283
music       Arts      4574
members     Arts      4111
yahoo       Arts      3590
dtype: int64

<IPython.core.display.Javascript object>